## PARCEIROS 360 - COMISSAO

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 09/12/2022
- <b>Descrição</b>: 

In [1]:
!python --version

Python 3.8.16


### BIBLIOTECAS:

In [2]:
# !pip install treeinterpreter

In [3]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [4]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [5]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### FUNÇÕES

In [6]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

### DADOS:

In [7]:
PATH = "datasets/"

***
### COMISSÃO: 

In [8]:
FILE = "02_COMISSAO.csv"
comissao = pd.read_csv(PATH + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
comissao = comissao[['RAIZ_CNPJ', 'COMPETENCIA']]

comissao['RAIZ_CNPJ'] = comissao['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
comissao = comissao[comissao["RAIZ_CNPJ"].str.contains("NAN")== False]
comissao = comissao[comissao["RAIZ_CNPJ"].str.contains("-")== False]

for i in comissao.columns:
    if comissao[i].dtypes=='object':
        comissao[i] = comissao[i].str.upper()
        
comissao = comissao.drop_duplicates().reset_index(drop=True)

comissao

,RAIZ_CNPJ,COMPETENCIA
0,00063960,2021-01-01
1,00066716,2021-01-01
2,00078599,2021-01-01
3,00244116,2021-01-01
4,00296549,2021-01-01
...,...,...
11300,93720464,2022-07-01
11301,96418264,2022-07-01
11302,96428529,2022-07-01
11303,96662168,2022-07-01


In [9]:
len(comissao['RAIZ_CNPJ'].unique())

850

***
### BASE DE CADASTRO DE FORNECEDORES:

In [10]:
FILE = "nome_fornecedores.xlsx"
nome_fornecedores = pd.read_excel(PATH + FILE)
nome_fornecedores = nome_fornecedores[['RAIZ_CNPJ', 'NOME_FORNECEDOR', 'FORNECEDOR', 'AREA_ATUACAO']]

nome_fornecedores['RAIZ_CNPJ'] = nome_fornecedores['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
nome_fornecedores = nome_fornecedores[nome_fornecedores["RAIZ_CNPJ"].str.contains("NAN")== False]
nome_fornecedores = nome_fornecedores[nome_fornecedores["RAIZ_CNPJ"].str.contains("-")== False]

for i in nome_fornecedores.columns:
    if nome_fornecedores[i].dtypes=='object':
        nome_fornecedores[i] = nome_fornecedores[i].str.upper()
        
nome_fornecedores = nome_fornecedores.drop_duplicates().reset_index(drop=True)

nome_fornecedores

,RAIZ_CNPJ,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO
0,02714643,333 COMERCIO E COMUNICACOES LTDA,7700312,COMÃRCIO
1,02714643,333 COMERCIO E COMUNICACOES LTDA,7700251,COMÃRCIO
2,02714643,333 COMERCIO E COMUNICACOES LTDA,7716773,COMÃRCIO
3,02714643,333 COMERCIO E COMUNICACOES LTDA,7721785,COMÃRCIO
4,02714643,333 COMERCIO E COMUNICACOES LTDA,7729957,COMÃRCIO
...,...,...,...,...
4572,08747596,ZOTTY CELULARES LTDA - ME,7729259,COMÃRCIO
4573,08747596,ZOTTY CELULARES LTDA - ME,7729332,COMÃRCIO
4574,05216804,ZTE DO BRASIL INDÃSTRIA COMÃRCIO,14017703,TELECOMUNICAÃÃES
4575,05216804,ZTE DO BRASIL INDÃSTRIA COMÃRCIO,14025498,TELECOMUNICAÃÃES


In [11]:
len(nome_fornecedores['RAIZ_CNPJ'].unique())

1319

***
### KPI

In [12]:
FILE = "20221017 - TBL_QUANT_COMISSAO.csv"
kpi = pd.read_csv(PATH + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
# kpi = pd.read_csv('/kaggle/input/datasetsnew/20221017 - TBL_QUANT_COMISSAO.csv', sep = ';', encoding = 'ISO-8859-1', engine ='python')
kpi = kpi[['RAIZ_CNPJ', 'INDICADOR_RISCO_FIN', 'FATOR_RISCO', 'RISCO_PARCEIRO']]
kpi['RAIZ_CNPJ'] = np.int64(kpi['RAIZ_CNPJ'])

kpi['RAIZ_CNPJ'] = kpi['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
kpi = kpi[kpi["RAIZ_CNPJ"].str.contains("NAN")== False]
kpi = kpi[kpi["RAIZ_CNPJ"].str.contains("-")== False]

for i in kpi.columns:
    if kpi[i].dtypes=='object':
        kpi[i] = kpi[i].str.upper()
        
kpi = kpi.drop_duplicates().reset_index(drop=True)

kpi

,RAIZ_CNPJ,INDICADOR_RISCO_FIN,FATOR_RISCO,RISCO_PARCEIRO
0,05970906,BAIXO RISCO,MELHORIA CONTÍNUA NA RECEITA,RISCO BAIXO
1,03231720,BAIXO RISCO,MELHORIA CONTÍNUA NA RECEITA,RISCO BAIXO
2,00308055,BAIXO RISCO,DETERIORAÇÃO RECENTE NA RECEITA,RISCO BAIXO
3,02183776,BAIXO RISCO,DETERIORAÇÃO RECENTE NA RECEITA,RISCO BAIXO
4,06368199,MÉDIO RISCO,MELHORA NO CURTO PRAZO APENAS,RISCO BAIXO
...,...,...,...,...
1230,32514823,ALTO RISCO,DÍVIDA TRABALHISTA ELEVADA,RISCO MÉDIO
1231,25161724,NaN,NaN,RISCO ALTO
1232,29320401,NaN,NaN,NÃO AVALIADO
1233,17299844,BAIXO RISCO,MELHORIA DE MÉDIO A LONGO PRAZO NA RECEITA,RISCO BAIXO


In [13]:
len(kpi['RAIZ_CNPJ'].unique())

850

***
### Join Comissão com KPI

In [14]:
comissao_kpi = comissao.join(kpi.set_index('RAIZ_CNPJ'),
                             how='inner',
                             on=['RAIZ_CNPJ'])\
                       .drop_duplicates().reset_index(drop=True)

comissao_kpi

,RAIZ_CNPJ,COMPETENCIA,INDICADOR_RISCO_FIN,FATOR_RISCO,RISCO_PARCEIRO
0,00063960,2021-01-01,MÉDIO RISCO,MELHORA NO CURTO PRAZO APENAS,RISCO MÉDIO
1,00063960,2021-02-01,MÉDIO RISCO,MELHORA NO CURTO PRAZO APENAS,RISCO MÉDIO
2,00063960,2021-03-01,MÉDIO RISCO,MELHORA NO CURTO PRAZO APENAS,RISCO MÉDIO
3,00063960,2021-06-01,MÉDIO RISCO,MELHORA NO CURTO PRAZO APENAS,RISCO MÉDIO
4,00063960,2021-08-01,MÉDIO RISCO,MELHORA NO CURTO PRAZO APENAS,RISCO MÉDIO
...,...,...,...,...,...
17508,46298215,2022-07-01,NaN,NaN,NÃO AVALIADO
17509,46337103,2022-06-01,NaN,NaN,NÃO AVALIADO
17510,46337103,2022-07-01,NaN,NaN,NÃO AVALIADO
17511,46354697,2022-06-01,NaN,NaN,NÃO AVALIADO


In [15]:
len(comissao_kpi['RAIZ_CNPJ'].unique())

850

***
### Join Comissão com NOME DE FORNECEDORES

In [16]:
comissao_forn = comissao_kpi.join(nome_fornecedores.set_index('RAIZ_CNPJ'),
                                 how='inner',
                                 on=['RAIZ_CNPJ'])\
                            .drop_duplicates().reset_index(drop=True)

comissao_forn['COMPETENCIA'] = pd.to_datetime(comissao_forn['COMPETENCIA'])

comissao_forn

,RAIZ_CNPJ,COMPETENCIA,INDICADOR_RISCO_FIN,FATOR_RISCO,RISCO_PARCEIRO,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO
0,00078599,2021-01-01,NaN,NaN,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS
1,00078599,2021-01-01,NaN,NaN,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS
2,00078599,2021-01-01,NaN,NaN,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7728224,RECURSOS HUMANOS
3,00078599,2021-01-01,NaN,NaN,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS
4,00078599,2021-01-01,NaN,NaN,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS
...,...,...,...,...,...,...,...,...
82669,46298215,2022-07-01,NaN,NaN,NÃO AVALIADO,SOUZA E SOUZA INTER. EM TELEFONIA L,7733983,SERVIÃOS
82670,46337103,2022-06-01,NaN,NaN,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES
82671,46337103,2022-07-01,NaN,NaN,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES
82672,46354697,2022-06-01,NaN,NaN,NÃO AVALIADO,ALLIANCE TELECOMUNICACOES BRASIL LT,7733964,COMÃRCIO


In [17]:
len(comissao_forn['RAIZ_CNPJ'].unique())

710

In [18]:
# fill em colunas com NaN
comissao_forn['INDICADOR_RISCO_FIN'] = comissao_forn['INDICADOR_RISCO_FIN'].fillna(value='SEM INDICADOR RISCO')
comissao_forn['FATOR_RISCO'] = comissao_forn['FATOR_RISCO'].fillna(value='SEM FATOR RISCO')
comissao_forn

,RAIZ_CNPJ,COMPETENCIA,INDICADOR_RISCO_FIN,FATOR_RISCO,RISCO_PARCEIRO,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO
0,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS
1,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS
2,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7728224,RECURSOS HUMANOS
3,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS
4,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS
...,...,...,...,...,...,...,...,...
82669,46298215,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,SOUZA E SOUZA INTER. EM TELEFONIA L,7733983,SERVIÃOS
82670,46337103,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES
82671,46337103,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES
82672,46354697,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,ALLIANCE TELECOMUNICACOES BRASIL LT,7733964,COMÃRCIO


In [19]:
comissao_forn = comissao_forn[comissao_forn["RAIZ_CNPJ"].str.contains("NAN")== False]
comissao_forn = comissao_forn[comissao_forn["RAIZ_CNPJ"].str.contains("-")== False]
comissao_forn = comissao_forn.drop_duplicates().reset_index(drop=True)
comissao_forn

,RAIZ_CNPJ,COMPETENCIA,INDICADOR_RISCO_FIN,FATOR_RISCO,RISCO_PARCEIRO,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO
0,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS
1,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS
2,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7728224,RECURSOS HUMANOS
3,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS
4,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS
...,...,...,...,...,...,...,...,...
82669,46298215,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,SOUZA E SOUZA INTER. EM TELEFONIA L,7733983,SERVIÃOS
82670,46337103,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES
82671,46337103,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES
82672,46354697,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,ALLIANCE TELECOMUNICACOES BRASIL LT,7733964,COMÃRCIO


In [20]:
from sklearn.preprocessing import OrdinalEncoder

comissao_forn = comissao_forn.rename(columns={'RISCO_PARCEIRO': 'KPI_comissao'})

comissao_forn_oe = OrdinalEncoder().fit_transform(comissao_forn[['INDICADOR_RISCO_FIN','FATOR_RISCO']])

comissao_forn['INDICADOR_RISCO_FIN_OE'] = comissao_forn_oe[:, 0]

comissao_forn['FATOR_RISCO_OE'] = comissao_forn_oe[:, 1]

comissao_forn

,RAIZ_CNPJ,COMPETENCIA,INDICADOR_RISCO_FIN,FATOR_RISCO,KPI_comissao,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO,INDICADOR_RISCO_FIN_OE,FATOR_RISCO_OE
0,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS,3.0,9.0
1,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS,3.0,9.0
2,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO BAIXO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7728224,RECURSOS HUMANOS,3.0,9.0
3,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7724567,RECURSOS HUMANOS,3.0,9.0
4,00078599,2021-01-01,SEM INDICADOR RISCO,SEM FATOR RISCO,RISCO MÉDIO,GERENCIAL BRASIL PARTICIPAÃÃES E EM,7726781,RECURSOS HUMANOS,3.0,9.0
...,...,...,...,...,...,...,...,...,...,...
82669,46298215,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,SOUZA E SOUZA INTER. EM TELEFONIA L,7733983,SERVIÃOS,3.0,9.0
82670,46337103,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES,3.0,9.0
82671,46337103,2022-07-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,R E COM E SER DE TELEFONIA LTDA,7733998,TELECOMUNICAÃÃES,3.0,9.0
82672,46354697,2022-06-01,SEM INDICADOR RISCO,SEM FATOR RISCO,NÃO AVALIADO,ALLIANCE TELECOMUNICACOES BRASIL LT,7733964,COMÃRCIO,3.0,9.0


In [21]:
comissao_forn.to_excel(r'intermediate/etl_previsao_comissao.xlsx')